In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
import re
import pandas as pd
import numpy as np

# Set Chrome options and the user-agent
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

# Initialize the ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)


origin = "SCL"
destination = "MXP"
startdate = "2023-12-11"
enddate = "2023-11-14"
url = f"https://www.kayak.cl/flights/{origin}-{destination}/{startdate}?sort=bestflight_a"


# Open the Kayak website
driver.get(url)
# sleep(13)
######################################################################################

# Presionar multiples veces 'ver mas resultaods' para obtener la mayor cantidad de vuelos posibles por búsqueda
while True:
    try:
        boton = WebDriverWait(driver, 25).until(EC.element_to_be_clickable((By.XPATH, '//div[@role="button" and @class="ULvh-button show-more-button"]')))
        boton.click()
    except:
        print("Button disappeared or not clickable. Stopping.")
        break

#######################################################################################
soup=BeautifulSoup(driver.page_source, 'html.parser')
# Extraer Horarios de salida y llegada
deptime = []
arrtime = []
div_elements_times = soup.find_all('div', class_='vmXl vmXl-mod-variant-large')

for div in div_elements_times:
    spans = div.find_all('span')
    if len(spans) == 3:
       deptime.append(spans[0].get_text())
       arrtime.append(spans[2].get_text())


# Extraer todos los precios
prices_int = []
div_elements_price = soup.find_all('div', class_='f8F1-price-text')

for div in div_elements_price:
    price_text = div.get_text()
    price_text = price_text.replace('$', '').replace('.', '')
    prices_int.append(int(price_text))


# Extraer lineas aereas
lineas = []
div_elements_lineas= soup.find_all('div', class_='J0g6-operator-text')

for div in div_elements_lineas:
    lineas_text = div.get_text()
    lineas.append(lineas_text)


# Extraer lineas aereas
escalas = []
span_elements_escalas= soup.find_all('span', class_='JWEO-stops-text')

for div in span_elements_escalas:
    escalas_text = div.get_text()
    escalas.append(escalas_text)


# Extraer aeropuertos de escalas
cleaned_airport_codes = []
div_elements_jweo = soup.find_all('div', class_='JWEO')

for div_jweo in div_elements_jweo:
    div_airport = div_jweo.find('div', class_='c_cgF c_cgF-mod-variant-full-airport')
    span_elements = div_airport.find_all('span')
    airport_codes = [span.get_text() for span in span_elements]
    cleaned_airport_code = ', '.join(set(filter(None, airport_codes)))
    cleaned_airport_codes.append(cleaned_airport_code)
cleaned_airport_codes = [', '.join(set(code.split(', '))) for code in cleaned_airport_codes]


#Extraer horas del viaje 
horas_viaje=[]
div_elements_horas = soup.find_all('div', class_='xdW8 xdW8-mod-full-airport')

for div in div_elements_horas:
    horas_text = div.find('div', class_='vmXl vmXl-mod-variant-default').get_text()
    horas_viaje.append(horas_text)


#Extraer tipo de clase
tipo_clase =[]
div_M_JD = soup.find_all('div', class_='M_JD')

for div in div_M_JD:
    div_aC3z_links = div.find('div', class_='aC3z-links')
    div_aC3z_option = div_aC3z_links.find('div', class_='aC3z-option')
    div_aC3z_name = div_aC3z_option.find('div', class_='aC3z-name').get_text()
    tipo_clase.append(div_aC3z_name)

# tipo_clase =[]
# div_M_JD = soup.find_all('div', class_='aC3z-name')
# for div in div_M_JD:
#     tipo_text= div.get_text()
#     tipo_clase.append(tipo_text)



Button disappeared or not clickable. Stopping.



KeyboardInterrupt



### Insertar variables al Data Frame

In [138]:
import pandas as pd

df = pd.DataFrame({"origin" : origin,
                    "destination" : destination,
                    "startdate" : startdate,
                    "linea_aerea": lineas,
                    "currency": "CLP",
                    "price": prices_int,
                    "deptime_o": deptime,
                    "arrtime_d": arrtime,
                    "escalas": escalas,
                    "COD_airport_escalas": cleaned_airport_codes,
                    "horas_viaje": horas_viaje,
                    "tipo_clase": tipo_clase
                    
                    })

In [145]:
df

,origin,destination,startdate,linea_aerea,currency,price,deptime_o,arrtime_d,escalas,COD_airport_escalas,horas_viaje,tipo_clase
0,SCL,MXP,2023-12-11,Air France,CLP,617444,12:00,10:35+1,1 escala,CDG,18h 35m,Light
1,SCL,MXP,2023-12-11,Vueling,CLP,439602,23:20,21:00+1,1 escala,BCN,17h 40m,Turista
2,SCL,MXP,2023-12-11,"LATAM Airlines, Air Europa",CLP,407666,17:15,9:15+2,2 escalas,"GRU, , MAD",36h 00m,Light
3,SCL,MXP,2023-12-11,LATAM Airlines,CLP,549120,12:17,9:20+1,1 escala,GRU,17h 03m,Light
4,SCL,MXP,2023-12-11,"LATAM Airlines, Lufthansa",CLP,461394,17:15,17:40+1,2 escalas,"GRU, , FRA",20h 25m,Light
...,...,...,...,...,...,...,...,...,...,...,...,...
105,SCL,MXP,2023-12-11,"LATAM Airlines, Iberia",CLP,2234429,16:40,18:25+1,2 escalas,"MAD, , LIM",21h 45m,Premium Business
106,SCL,MXP,2023-12-11,British Airways,CLP,2243001,10:57,16:40+1,2 escalas,"GRU, , LHR",25h 43m,Turista superior
107,SCL,MXP,2023-12-11,"KLM, Lufthansa",CLP,7984172,12:40,17:40+1,2 escalas,", EZE, FRA",25h 00m,Ejecutiva
108,SCL,MXP,2023-12-11,"KLM, Lufthansa",CLP,7984172,12:40,17:40+1,2 escalas,", EZE, FRA",25h 00m,Ejecutiva


### Revisar base de datos SQLITE

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('master.db')
vuelos = 'vuelos'  # Reemplaza 'nombre_de_la_tabla' con el nombre de la tabla que deseas extraer
df1 = pd.read_sql_query(f"SELECT * FROM {vuelos}", conn)
conn.close()
df1

,Origin,Destination,StartDate,DepartureTime,ArrivalTime,Price,Airline,Stops,LayoverAirports,TravelTime,ClassType
0,SCL,ATL,2023-12-11,5:10,10:15+1,257997,"Copa Airlines, Frontier",2 escalas,", PTY, MIA",31h 05m,Turista
1,SCL,ATL,2023-12-11,23:15,13:37+1,346180,"LATAM Airlines, Frontier",1 escala,JFK-LGA,16h 22m,Basic
2,SCL,ATL,2023-12-11,5:10,10:15+1,257997,"Copa Airlines, Frontier",2 escalas,", PTY, MIA",31h 05m,Turista
3,SCL,ATL,2023-12-11,20:55,10:15+1,315902,"LATAM Airlines, Frontier",2 escalas,", LIM, MIA",15h 20m,Light
4,SCL,ATL,2023-12-11,23:20,6:55+1,891353,LATAM Airlines,directo,,9h 35m,Light
...,...,...,...,...,...,...,...,...,...,...,...
1924,SCL,CCS,2023-12-25,20:55,12:09+1,1012668,"LATAM Airlines, Copa Airlines",3 escalas,"BOG, , LIM, ...",16h 14m,Basic
1925,SCL,CCS,2023-12-25,6:30,1:21+1,1084461,Varias aerolíneas,3 escalas,"BOG, , LIM, ...",19h 51m,Basic
1926,SCL,CCS,2023-12-25,6:45,1:21+1,1343213,"Avianca, Copa Airlines",2 escalas,"BOG, PTY,",19h 36m,Economy XS
1927,SCL,CCS,2023-12-25,20:55,14:57+1,1012668,"LATAM Airlines, Copa Airlines",3 escalas,"BOG, , LIM, ...",19h 02m,Basic


### Chequeo de valores destinos únicos

In [17]:
df2 = df1['Destination'].unique().tolist()
print(df2)



['ATL', 'DEN', 'SAN', 'SFO', 'JFK', 'BOS', 'IAD', 'MIA', 'PAR', 'CCS']
